# Movie recommender system 

### User-based and Item-based Collaborative Filtering methods

In [1]:
import pandas as pd
from scipy.stats import pearsonr
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")

In [2]:
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [3]:
# helper functions

# given userid and movieid, get what the user rated the movie
def getMovieRating(user, movie):
    return (ratings.loc[(ratings['userId']==user) & (ratings['movieId']==movie),'rating'].iloc[0])

# given userid, get what movies that user has rated
def getAllMoviesRated(user):
    return (ratings.loc[(ratings['userId']==user),'movieId'].tolist())

# given userid, get what movies that user has rated above certain rating
def getTopRatedMovies(user, rating):
    return (ratings.loc[(ratings['userId']==user) & (ratings['rating']>rating) ,'movieId'].tolist())

# give userid, get one of their highest rated movies
def getTopRatedMovie(user):
    return (ratings.loc[(ratings['userId']==user) & (ratings['rating']==5),'movieId'].iloc[0])

# get the movie name by movie id
def getMovieName(movieId):
    return (movies.loc[(movies['movieId']==movieId),'title']).iloc[0]

# get the user ratings for given movie from everyone who has watched it except targeted user(s)
def getUserRatingsForMovie(movie, users):
    return (ratings.loc[(ratings['movieId']==movie) & (~ratings['userId'].isin(users)), 'rating'].tolist())

# get the user ratings for given movie from all given users who have watched it
def getUserRatingsForMovieWatched(movie, users):
    return (ratings.loc[(ratings['movieId']==movie) & (ratings['userId'].isin(users)), 'rating'].tolist())


In [6]:
def users_similarity(user1, user2):
    
    moviesUser1 = getAllMoviesRated(user1)
    moviesUser2 = getAllMoviesRated(user2)
    
    moviesBothUsers = []
    userRatings1 = []
    userRatings2 = []
    
    # find movies that both users have watched
    for movie in moviesUser1:
        if movie in moviesUser2:
            moviesBothUsers.append(movie)
    
    # if the users have no movies in common
    if len(moviesBothUsers) == 0:
        return 0
    
    # create ratings list for common movies between both users
    for movie in moviesBothUsers:
        userRatings1.append(getMovieRating(user1,movie))
        userRatings2.append(getMovieRating(user2,movie))
    
    # to see how similar two users are:
    # find pearson correlation similarity coefficient for ratings between the two users for the same movies
    corr, p_value = pearsonr(userRatings1, userRatings2)
    
    return corr

In [8]:
def findUserBasedRecommendations(user10):
    
    moviesSeenByUser10 = getAllMoviesRated(user10) # all movies given user has seen
    recommendations = []
    recMovies = []
    
    # get all unique users from the ratings df
    allUsers = ratings.userId.unique().tolist()
    
    for user in allUsers:
        if user != user10: # don't compare user to itself
            
            # get the pearson correlation score between given user and each of the other users
            score = users_similarity(user10,user) 
            
            if score > 0: # ignore user-user pair if score is negative - weak relationship similarity wise
                moviesSeen = getAllMoviesRated(user) # all movies the other user has seen
                for movie in moviesSeen:
                    movieName = getMovieName(movie) # get name from id
                    movieScore = (movieName,score) # movie name and similarity score
#                     movieScore = (movie,score)
                
                    # ignore if given user has already seen movie and keep list unique
                    if movie not in moviesSeenByUser10 and movie not in recMovies: 
                        recommendations.append(movieScore)
                        recMovies.append(movie)
        
    # sort by similarity scores in descending order
    recommendations = sorted(recommendations, key=lambda pair: pair[1], reverse=True)
    
    return recommendations[:20] # return top 5 movies

## Top 5 recommended movies for user 10 (user-based collaborative filtering):

In [9]:
findUserBasedRecommendations(10)

[('Friday (1995)', 1.0),
 ('Eraser (1996)', 1.0),
 ('Jade (1995)', 1.0),
 ('Johnny Mnemonic (1995)', 1.0),
 ('Prophecy, The (1995)', 1.0),
 ('Strange Days (1995)', 1.0),
 ('Jerky Boys, The (1995)', 1.0),
 ('Murder in the First (1995)', 1.0),
 ('Tales from the Crypt Presents: Demon Knight (1995)', 1.0),
 ('Virtuosity (1995)', 1.0),
 ('Hellraiser: Bloodline (1996)', 1.0),
 ('Operation Dumbo Drop (1995)', 1.0),
 ('Hamlet (1996)', 1.0),
 ('Flatliners (1990)', 1.0),
 ('102 Dalmatians (2000)', 1.0),
 ('3-Iron (Bin-jip) (2004)', 1.0),
 ('21 (2008)', 1.0),
 ('Sudden Death (1995)', 1.0),
 ('Great White Hype, The (1996)', 1.0),
 ('Cemetery Man (Dellamorte Dellamore) (1994)', 1.0)]

In [6]:
def movies_similarity(movie1, movie2):
    
    usersSeenMovie1 = []
    usersSeenMovie1And2 = []
    
    # get all unique users from the ratings df
    allUsers = ratings.userId.unique().tolist()
    
    # get all users who have seen movie 1
    for user in allUsers:
        try: 
            rating = getMovieRating(user,movie1)  # user has rated movie1
            usersSeenMovie1.append(user)
        except:   
            pass # user has not seen movie 1
    
    # get all users who have seen movie 1 and movie 2
    for user in usersSeenMovie1: 
        try:
            rating = getMovieRating(user,movie2) # user has seen both movie1 and movie2
            usersSeenMovie1And2.append(user)
        except:
            pass # user has only seen movie1
       
    # only do further similarity calculations if at least 50 same users have watched the same movie
    # to avoid getting strong positive associations between movies based just a few users data    
    if len(usersSeenMovie1And2) > 50: 
        # get the movie ratings for the two given movies among subset of those users who have seen both
        movie1Ratings = getUserRatingsForMovieWatched(movie1,usersSeenMovie1And2)
        movie2Ratings = getUserRatingsForMovieWatched(movie2,usersSeenMovie1And2)

        # to see how similar two movies are:
        # find pearson correlation similarity coefficient for ratings between the two movies for the same users
        corr, p_value = pearsonr(movie1Ratings, movie2Ratings)
    else:
        corr = 0    
    
    return corr

In [9]:
def findItemBasedRecommendations(user):
    allMovies = movies.movieId.unique().tolist() # all movies in dataset
    allUsers = ratings.userId.unique().tolist() # all users in dataset

    recMovies = []
    
    print("here")

    for user in allUsers:
        for mov in allMovies:
            movie = getTopRatedMovie(user) # get highest rated movie of the user
            print("movie: " + str(movie))
            
            if movie != mov and movie not in allMovies:
                
                print("mov " + str(mov))
                
                # get movie similarity score for each movie-movie pair between the list of movies already watched by user
                # and the overall top rated movies, not including movies in user's watched list
                score = movies_similarity(movie, mov) 
#                 print("movie", movie, " score", score)

                if score > 0:
                    movieName = getMovieName(movie) # get name from id

                    if movieName not in recMovies:  # if movie not already added to recommendations list
                        recMovies.append(movieName)
                        break # move on to compare with next movie on all movies list 

    return recMovies[:5] # return top 5 movies

In [ ]:
findItemBasedRecommendations(10)

here
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie

movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47


movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47


movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47


movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47
movie: 47


...

movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756

movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756

movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756

movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756

movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756
movie: 60756

## Top 5 recommended movies for user 10 (item-based collaborative filtering):

In [69]:
findItemBasedRecommendations(10)

['Seven (a.k.a. Se7en) (1995)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Usual Suspects, The (1995)',
 'Shawshank Redemption, The (1994)',
 'Shrek 2 (2004)']

### The two methods returned very different results from each other. The user-based way works better in the context of performance and how easy it was to implement. But I feel item-based filtering renders the more accurate results because the amount of data is huge and so the idea of user to user similarity loses it meaning when there are these many movies.